In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow import keras
import random
import cv2
import math
from PIL import Image
import seaborn as sns
import csv
import json
from tqdm import tqdm

from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten
from tensorflow.python.keras.saving.hdf5_format import save_attributes_to_hdf5_group

import joblib

In [2]:
model = Sequential()
model.add(Conv2D(32, (4, 4), padding="same", activation='relu', input_shape=(16, 16, 1)))
model.add(MaxPooling2D((4, 4)))
model.add(Conv2D(64, (2, 2), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(16, activation='softmax'))

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [3]:
label_dict = {'fivefingerR': 0, 'fourfingerR': 1, 'threefingerR': 2, 'twofingerR': 3, 'onefingerR': 4,
              'fistR': 5, 'cowabungaR': 6, 'spidermanR': 7, 'okR': 8, 'cR': 9, 'thumbR': 10,
              'lR': 11, 'oR': 12, 'rockonR': 13, 'palmR': 14, 'sidehandR': 15}

index_dict = {value : key for key, value in label_dict.items()}

with open('data_short.json', 'r') as file:
    raw_data = json.load(file)

labels = []
data = []

for label, arrays in raw_data.items():
    for array in arrays:
        def_label = np.zeros((16))
        def_label[label_dict[label]] = 1
        labels.append(def_label)
        normalizedData = np.array((array-np.min(array))/(np.max(array)-np.min(array)))
        data.append(normalizedData.reshape(16, 16, 1))

labels = np.array(labels)
data = np.array(data)

(xtrain,xtest,ytrain,ytest)=train_test_split(data,labels,test_size=0.4)
# print(xtrain.shape, xtest.shape)


In [4]:
model_t = model.fit(xtrain, ytrain, epochs=20, batch_size = 64, verbose=2, validation_data=(xtest, ytest))

Epoch 1/20
30/30 - 0s - loss: 2.7643 - accuracy: 0.0891 - val_loss: 2.7515 - val_accuracy: 0.0578 - 461ms/epoch - 15ms/step
Epoch 2/20
30/30 - 0s - loss: 2.7195 - accuracy: 0.1724 - val_loss: 2.6790 - val_accuracy: 0.2414 - 194ms/epoch - 6ms/step
Epoch 3/20
30/30 - 0s - loss: 2.5615 - accuracy: 0.2766 - val_loss: 2.4017 - val_accuracy: 0.2711 - 149ms/epoch - 5ms/step
Epoch 4/20
30/30 - 0s - loss: 2.1570 - accuracy: 0.4161 - val_loss: 1.9453 - val_accuracy: 0.4164 - 182ms/epoch - 6ms/step
Epoch 5/20
30/30 - 0s - loss: 1.6847 - accuracy: 0.5693 - val_loss: 1.4817 - val_accuracy: 0.6438 - 159ms/epoch - 5ms/step
Epoch 6/20
30/30 - 0s - loss: 1.3350 - accuracy: 0.6599 - val_loss: 1.2014 - val_accuracy: 0.6984 - 141ms/epoch - 5ms/step
Epoch 7/20
30/30 - 0s - loss: 1.0861 - accuracy: 0.7479 - val_loss: 0.9542 - val_accuracy: 0.8039 - 139ms/epoch - 5ms/step
Epoch 8/20
30/30 - 0s - loss: 0.8887 - accuracy: 0.7948 - val_loss: 0.7770 - val_accuracy: 0.8211 - 139ms/epoch - 5ms/step
Epoch 9/20
30/3

In [6]:
# loaded_rf = joblib.load("model.joblib")

def getTotalAccuracy(model):
    with open('./data/data.json', 'r') as file:
        data = json.load(file)

    correct = np.zeros((16))
    incorrect = np.zeros((16))
    total = np.zeros((16))

    for label, arrays in (data.items()):
        print(label, len(arrays))
        for image in tqdm(arrays):
            input_data = np.reshape(image, (-1, 16, 16, 1))
            predictions = np.squeeze(model.predict(input_data, verbose=0))
            max_idx = predictions.tolist().index(max(predictions))
            label_out = index_dict[max_idx]
            
            if (label == label_out):
                correct[index_dict[label]] += 1
            else:
                incorrect[index_dict[label]] += 1
            total[index_dict[label]] += 1

    print(f"Total samples: {np.sum(total)}\nTotal correct: {np.sum(correct)}\nTotal incorrect: {np.sum(incorrect)}")
    print(f"Accuracy: {np.sum(correct) / np.sum(total)}\n")
    print("Number Correct by Index:\n", correct)
    print("Number Incorrect by Index:\n", incorrect)
    print("Accuracy by Index:\n", correct / total)


getTotalAccuracy(model)



# # Load the image
# image_path = "data_pad.jpg"
# image = load_img(image_path, target_size=(32, 32))  # Resize the image to match the input size of the CNN
# image_array = img_to_array(image)  # Convert the image to a numpy array
# expanded_image_array = tf.expand_dims(image_array, axis=0)  # Expand the dimensions to match the batch size

# # Preprocess the image
# preprocessed_image = preprocess_input(expanded_image_array)

# # Load the pre-trained CNN model
# model = model_d

# # Make predictions on the image
# predictions = model.predict(preprocessed_image)

  0%|          | 0/16 [00:00<?, ?it/s]

fivefingerR 1386


  6%|▋         | 1/16 [00:38<09:41, 38.78s/it]

fourfingerR 1144


 12%|█▎        | 2/16 [01:09<07:58, 34.18s/it]

...

palmR 1221


 94%|█████████▍| 15/16 [10:05<00:38, 38.95s/it]

sidehandR 1184


100%|██████████| 16/16 [10:41<00:00, 40.08s/it]


Total samples: 21780
Total correct: 13447
Total incorrect: 8333
Accuracy: 0.6174012855831038



In [7]:
# frequency ranges from A3 to C5
freqs = [220, 233, 247, 262, 277, 294, 311, 330, 349, 370, 392, 415, 440, 466, 494, 523]
freqDict = {key : f for (key, f) in (index_dict.keys(), freqs)}
for key in index_dict.keys():
    freqDict

print(freqDict)

ValueError: too many values to unpack (expected 2)